![RAGA流程](.RAG流程图.png)

## 1.数据抓取/采集

In [1]:
import requests
from langchain.document_loaders import TextLoader
# 指定要抓取的URL
url = "https://consumer.huawei.com/cn/laptops/matebook-gt-14/"

# 发送GET请求并获取响应
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
}
res = requests.get(url, headers=headers)

# 检查响应状态码是否为200（成功）
if res.status_code == 200:
    # 将响应内容写入本地文件
    with open("raw_data/huawei.html", "w") as f:
        f.write(res.text)
else:
    print(f"请求失败，状态码: {res.status_code}")

loader = TextLoader('raw_data/huawei.html')
documents = loader.load()

## 2.文档分块

In [ ]:
from langchain.text_splitter import CharacterTextSplitter # 用于文本分割

# 实例化CharacterTextSplitter类，设置文本块大小和重叠度
# chunk_size定义每个文本块的大小为500个字符
# chunk_overlap定义相邻文本块之间的重叠字符数为50，以保持内容连贯性
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# 使用text_splitter分割文档，documents变量应包含待处理的文本数据
# split_documents方法将文档分割成多个文本块，便于后续处理
chunks = text_splitter.split_documents(documents)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

# 创建一个嵌入式Weaviate客户端，用于在本地运行Weaviate而无需外部服务
client = weaviate.Client(url="http://localhost:8080",embedded_options=EmbeddedOptions())

# 使用Weaviate.from_documents方法从文档集合中创建向量存储
vectorstore = Weaviate.from_documents(
    client = client,                # client: 指定Weaviate客户端
    documents = chunks,             # documents: 指定要存储的文档集合，此处为变量chunks，需在外部定义
    embedding = OpenAIEmbeddings(), # embedding: 指定文本嵌入模型，此处使用OpenAIEmbeddings
    by_text = False                 # by_text: 指定是否将文档拆分为文本块进行处理，False表示不拆分
)

## 3.RAG流程

### 3.1 检索
首先，需要在向量数据库中填充数据，然后定义一个检索器组件。这个检索器会根据用户的查询和嵌入块之间的语义相似度，为我们提供额外的上下文信息。

In [ ]:
retriever = vectorstore.as_retriever()

### 3.2 增强
接下来，我们需要使用检索到的上下文信息来增强我们的提示。为此，我们需要准备一个提示模板。在LangChain中，我们可以使用ChatPromptTemplate来创建一个提示模板。这个模板会告诉LLM如何使用检索到的上下文来回答问题。如果LLM不知道答案，它会说出“我不知道”。我们要求LLM的答案尽可能简洁，最多使用三个句子。在Python中，我们可以这样实现：

In [ ]:
from langchain.prompts import ChatPromptTemplate
 
template = """你是问答任务助手。使用以下检索到的上下文片段来回答问题。如果你不知道答案，就说你不知道。最多使用三个句子，保持答案简洁。
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)
 
print(prompt)

### 3.3 生成
最后，我们需要构建一个RAG流程链条，将检索器、提示模板和LLM连接起来。定义好RAG链后，我们就可以调用它进行生成。

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# 初始化OpenAI的聊天模型，使用GPT-3.5-turbo模型，设置温度为0以获取确定性响应
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 构建一个链式处理结构用于处理输入问题并生成回答
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)
# 定义一个查询，询问关于最新华为Matebook的创新功能 
query = "最新的华为Matebook有哪些创新功能？"
# 使用构建好的链式处理结构处理查询，生成回答
rag_chain.invoke(query)